In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import time
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor
from nltk.stem.snowball import SnowballStemmer
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings('ignore')

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input/"))
import numba

# Any results you write to the current directory are saved as output.

['feature_test.csv', 'custom.css', '__results__.html', 'feature_train.csv', 'product_expand.csv', 'df_all_toeknized.csv', '__output__.json', '__notebook__.ipynb']


In [2]:
df_all = pd.read_csv('../input/df_all_toeknized.csv',sep = "\t")


In [3]:
df_all.head()

,attribute,id,product_description,product_title,product_uid,relevance,search_term,product_description_tokens,product_title_tokens,search_term_tokens,attribute_tokens,product_description_stemmed,product_title_stemmed,search_term_stemmed,attribute_stemmed
0,Versatile connector for various 90 connections...,2,"Not only do angles make joints stronger, they ...",Simpson Strong-Tie 12-Gauge Angle,100001,3.00,angle bracket,not only do angles make joints stronger they a...,simpson strongtie 12gauge angle,angle bracket,versatile connector for various 90 connections...,not onli do angl make joint stronger they also...,simpson strongti angl,angl bracket,versatil connector for variou connect and home...
1,Versatile connector for various 90 connections...,3,"Not only do angles make joints stronger, they ...",Simpson Strong-Tie 12-Gauge Angle,100001,2.50,l bracket,not only do angles make joints stronger they a...,simpson strongtie 12gauge angle,l bracket,versatile connector for various 90 connections...,not onli do angl make joint stronger they also...,simpson strongti angl,l bracket,versatil connector for variou connect and home...
2,"Brush,Roller,Spray 6.63 in 7.76 in 6.63 in Rev...",9,BEHR Premium Textured DECKOVER is an innovativ...,BEHR Premium Textured DeckOver 1-gal. #SC-141 ...,100002,3.00,deck over,behr premium textured deckover is an innovativ...,behr premium textured deckover 1gal sc141 tugb...,deck over,brushrollerspray 663 in 776 in 663 in revives ...,behr premium textur deckov is an innov solid c...,behr premium textur deckov tugboat wood and co...,deck over,brushrollerspray in in in reviv wood and compo...
3,Combo Tub and Shower No Includes the trim kit ...,16,Update your bathroom with the Delta Vero Singl...,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,100005,2.33,rain shower head,update your bathroom with the delta vero singl...,delta vero 1handle shower only faucet trim kit...,rain shower head,combo tub and shower no includes the trim kit ...,updat your bathroom with the delta vero single...,delta vero shower onli faucet trim kit in chro...,rain shower head,combo tub and shower no includ the trim kit on...
4,Combo Tub and Shower No Includes the trim kit ...,17,Update your bathroom with the Delta Vero Singl...,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,100005,2.67,shower only faucet,update your bathroom with the delta vero singl...,delta vero 1handle shower only faucet trim kit...,shower only faucet,combo tub and shower no includes the trim kit ...,updat your bathroom with the delta vero single...,delta vero shower onli faucet trim kit in chro...,shower onli faucet,combo tub and shower no includ the trim kit on...


In [4]:
df_all.isnull().sum()

attribute                       43974
id                                  0
product_description                 0
product_title                       0
product_uid                         0
relevance                      112067
search_term                         0
product_description_tokens          0
product_title_tokens                0
search_term_tokens                  0
attribute_tokens                43974
product_description_stemmed         0
product_title_stemmed               0
search_term_stemmed               626
attribute_stemmed               43974
dtype: int64

## Some seacrch_term only contains numbers so they become null because I use isalpha to filter them. So I assign them with tokens

In [5]:
df_all.search_term_stemmed.fillna(df_all.search_term_tokens, inplace=True)

In [6]:
df_all.isnull().sum()

attribute                       43974
id                                  0
product_description                 0
product_title                       0
product_uid                         0
relevance                      112067
search_term                         0
product_description_tokens          0
product_title_tokens                0
search_term_tokens                  0
attribute_tokens                43974
product_description_stemmed         0
product_title_stemmed               0
search_term_stemmed                 0
attribute_stemmed               43974
dtype: int64

In [7]:
df_all.fillna("",inplace=True)

In [8]:
df_all.to_csv("df_all_toeknized_filled_na_search_term.csv",index = False)

In [9]:
df_all['cos_description'] = 0
df_all['cos_title'] = 0
df_all['cos_attribute'] = 0

In [10]:
def get_cosine_sim(*strs): 
    try:
        vectors = [t for t in get_vectors(*strs)]
        return cosine_similarity(vectors)[0][1]
    except:
        return 0
    
def get_vectors(*strs):
    text = [t for t in strs]
    vectorizer = CountVectorizer(text)
    vectorizer.fit(text)
    return vectorizer.transform(text).toarray()

def get_vectors_2(text):
    cv = CountVectorizer(max_features=5000)
    cv.fit(text)
    return cv.transform(text)


In [11]:
#df_all = df_all[:10]

df_all["cos_description_stemmed"] = df_all[["product_description_stemmed","search_term_stemmed"]].apply(lambda x:get_cosine_sim(*x),axis = 1) 
df_all["cos_title_stemmed"] = df_all[["product_title_stemmed","search_term_stemmed"]].apply(lambda x:get_cosine_sim(*x),axis = 1) 
df_all["cos_attribute_stemmed"] = df_all[["attribute_stemmed","search_term_stemmed"]].apply(lambda x:get_cosine_sim(*x),axis = 1) 

In [12]:
#df_all.to_csv("df_all_stemmed_cos_sim.csv",index = False)